In [1]:
# Hide warnings
from logging import FATAL
import warnings

warnings.filterwarnings('ignore')

# General dependencies
import json
import networkx as nx
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from typing import List, Tuple

# Qiskit global
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit import Aer, QuantumCircuit

# Qiskit Algorithms
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
    RecursiveMinimumEigenOptimizer,
    SolutionSample,
    OptimizationResultStatus,
)
from qiskit_optimization import QuadraticProgram

from qiskit.quantum_info import PauliTable
from qiskit.opflow.primitive_ops.primitive_op import PrimitiveOp
from qiskit.opflow.primitive_ops.pauli_sum_op import PauliSumOp
from qiskit.quantum_info.operators.symplectic.sparse_pauli_op import SparsePauliOp

# Qiskit Visuals
from qiskit.visualization import plot_histogram

# Custom libraries
import qaoa_vrp.build_graph
import qaoa_vrp.features.graph_features
import qaoa_vrp.features.tsp_features
import qaoa_vrp.build_circuit
import qaoa_vrp.clustering
import qaoa_vrp.utils
from qaoa_vrp.plot.feasibility_graph import (
    plot_feasibility_results,
    generate_feasibility_results,
)
from qaoa_vrp.initialisation.initialisation import Initialisation
from qaoa_vrp.features.graph_features import get_graph_features
from qaoa_vrp.features.tsp_features import get_tsp_features
from qaoa_vrp.parallel.optimize_qaoa import run_qaoa_parallel_control_max_restarts

plt.style.use('seaborn')
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [71]:
# SET GLOBAL VARIABLES
NUM_LAYERS = 1
INITIAL_POINT = Initialisation().trotterized_quantum_annealing(p = NUM_LAYERS)

RQAOA_MIN_NUM_VARS = 3
RQAOA_N_RESTARTS = 15
RQAOA_MIN_NUM_VARS = [1, 3, 5]

In [2]:
filename = "instanceType_asymmetric_tsp_numNodes_4_numVehicles_1_0083a1d22a6447f69091ac552ceb8ee2.json"
instance_path = "../../data/{}".format(filename)

In [3]:
with open(instance_path) as f:
    data = json.load(f)
    G, depot_info = qaoa_vrp.build_graph.build_json_graph(data["graph"])
    num_vehicles = int(data["numVehicles"])
    threshold = float(data["threshold"])
    n_max = int(data["n_max"])

In [4]:
edge_mat = nx.linalg.graphmatrix.adjacency_matrix(G).toarray()
cost_mat = np.array(nx.attr_matrix(G, edge_attr="cost", rc_order=list(G.nodes())))
for edge in G.edges():
    G[edge[0]][edge[1]]['cost'] = 0

edge_mat = nx.linalg.graphmatrix.adjacency_matrix(G).toarray()
cost_mat = np.array(nx.attr_matrix(G, edge_attr="cost", rc_order=list(G.nodes())))

G, cluster_mapping = qaoa_vrp.clustering.create_clusters(
    G, num_vehicles, "spectral-clustering", edge_mat
)

depot_edges = list(G.edges(depot_info["id"], data=True))
depot_node = depot_info["id"]

subgraphs = qaoa_vrp.clustering.build_sub_graphs(G, depot_node, depot_edges)

# big_offset = sum(sum(cost_mat))/2 + 1
big_offset = 30
qubos = qaoa_vrp.build_circuit.build_qubos(subgraphs, depot_info, A=big_offset)

cluster_mapping = [i + 1 for i in cluster_mapping]
cluster_mapping.insert(0, 0)

qubo = qubos[0]

In [5]:
op_tsp, offset = qubo.to_ising()
print("offset: {}".format(offset))
print("operator: {}".format(op_tsp))

offset: 180.0
operator: SummedOp([
  -30.0 * IIIIIIIIZ,
  -30.0 * IIIIIIIZI,
  -30.0 * IIIIIIZII,
  -30.0 * IIIIIZIII,
  -30.0 * IIIIZIIII,
  -30.0 * IIIZIIIII,
  -30.0 * IIZIIIIII,
  -30.0 * IZIIIIIII,
  -30.0 * ZIIIIIIII,
  15.0 * IIIIIIIZZ,
  15.0 * IIIIIIZIZ,
  15.0 * IIIIIIZZI,
  15.0 * IIIIIZIIZ,
  15.0 * IIIIZIIZI,
  15.0 * IIIIZZIII,
  15.0 * IIIZIIZII,
  15.0 * IIIZIZIII,
  15.0 * IIIZZIIII,
  15.0 * IIZIIIIIZ,
  15.0 * IIZIIZIII,
  15.0 * IZIIIIIZI,
  15.0 * IZIIZIIII,
  15.0 * IZZIIIIII,
  15.0 * ZIIIIIZII,
  15.0 * ZIIZIIIII,
  15.0 * ZIZIIIIII,
  15.0 * ZZIIIIIII
])


In [6]:
list_of_paulis = []
list_of_coeffs = []
for i in list(op_tsp):
    list_of_paulis.append(str(i.primitive))
    list_of_coeffs.append(i.coeff)

for i, pauli in enumerate(list_of_paulis):
    if i == 0:
        pauli_table = PauliTable(list_of_paulis[0])
    else:
        pauli_table += PauliTable(list_of_paulis[i])
        
op_tsp = PauliSumOp(SparsePauliOp(pauli_table, coeffs=list_of_coeffs))

In [7]:
qp = QuadraticProgram()
qp.from_ising(op_tsp, offset, linear=True)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 60 x0 - 60 x1 - 60 x2 - 60 x3 - 60 x4 - 60 x5 - 60 x6 - 60 x7 - 60 x8 +
      [ 120 x0*x1 + 120 x0*x2 + 120 x0*x3 + 120 x0*x6 + 120 x1*x2 + 120 x1*x4
      + 120 x1*x7 + 120 x2*x5 + 120 x2*x8 + 120 x3*x4 + 120 x3*x5 + 120 x3*x6
      + 120 x4*x5 + 120 x4*x7 + 120 x5*x8 + 120 x6*x7 + 120 x6*x8 + 120 x7*x8
      ]/2 + 180
Subject To

Bounds
 0 <= x0 <= 1
 0 <= x1 <= 1
 0 <= x2 <= 1
 0 <= x3 <= 1
 0 <= x4 <= 1
 0 <= x5 <= 1
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1

Binaries
 x0 x1 x2 x3 x4 x5 x6 x7 x8
End



In [100]:
algorithm_globals.random_seed = 10598
quantum_instance = QuantumInstance(
    Aer.get_backend("aer_simulator_matrix_product_state"),
    seed_simulator=algorithm_globals.random_seed,
    seed_transpiler=algorithm_globals.random_seed,
)
qaoa_mes = QAOA(quantum_instance=quantum_instance, reps=NUM_LAYERS, initial_point=INITIAL_POINT)
exact_mes = NumPyMinimumEigensolver()


In [101]:
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

In [10]:
qp

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 60 x0 - 60 x1 - 60 x2 - 60 x3 - 60 x4 - 60 x5 - 60 x6 - 60 x7 - 60 x8 +
      [ 120 x0*x1 + 120 x0*x2 + 120 x0*x3 + 120 x0*x6 + 120 x1*x2 + 120 x1*x4
      + 120 x1*x7 + 120 x2*x5 + 120 x2*x8 + 120 x3*x4 + 120 x3*x5 + 120 x3*x6
      + 120 x4*x5 + 120 x4*x7 + 120 x5*x8 + 120 x6*x7 + 120 x6*x8 + 120 x7*x8
      ]/2 + 180
Subject To

Bounds
 0 <= x0 <= 1
 0 <= x1 <= 1
 0 <= x2 <= 1
 0 <= x3 <= 1
 0 <= x4 <= 1
 0 <= x5 <= 1
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1

Binaries
 x0 x1 x2 x3 x4 x5 x6 x7 x8
End

In [11]:
ising_qubo = qubos[0].to_ising()

In [12]:
list_of_paulis = []
list_of_coeffs = []
for i in list(ising_qubo[0]):
    list_of_paulis.append(str(i.primitive))
    list_of_coeffs.append(i.coeff)


In [14]:
for i, pauli in enumerate(list_of_paulis):
    if i == 0:
        pauli_table = PauliTable(list_of_paulis[0])
    else:
        pauli_table += PauliTable(list_of_paulis[i])


In [15]:
op_tsp = PauliSumOp(SparsePauliOp(pauli_table, coeffs=list_of_coeffs))

In [16]:
qp = QuadraticProgram()
qp.from_ising(op_tsp, offset, linear=True)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 60 x0 - 60 x1 - 60 x2 - 60 x3 - 60 x4 - 60 x5 - 60 x6 - 60 x7 - 60 x8 +
      [ 120 x0*x1 + 120 x0*x2 + 120 x0*x3 + 120 x0*x6 + 120 x1*x2 + 120 x1*x4
      + 120 x1*x7 + 120 x2*x5 + 120 x2*x8 + 120 x3*x4 + 120 x3*x5 + 120 x3*x6
      + 120 x4*x5 + 120 x4*x7 + 120 x5*x8 + 120 x6*x7 + 120 x6*x8 + 120 x7*x8
      ]/2 + 180
Subject To

Bounds
 0 <= x0 <= 1
 0 <= x1 <= 1
 0 <= x2 <= 1
 0 <= x3 <= 1
 0 <= x4 <= 1
 0 <= x5 <= 1
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1

Binaries
 x0 x1 x2 x3 x4 x5 x6 x7 x8
End



In [41]:
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

In [46]:
exact_result = exact.solve(qp)
print(exact_result)

optimal function value: 0.0
optimal value: [0. 0. 1. 1. 0. 0. 0. 1. 0.]
status: SUCCESS


In [29]:
qaoa_result = qaoa.solve(qp)
print(qaoa_result)

optimal function value: 0.0
optimal value: [0. 0. 1. 0. 1. 0. 1. 0. 0.]
status: SUCCESS


In [66]:
# Identify probability of success
FEASIBLE_SOLUTIONS = [
    [1.,0.,0.,0.,1.,0.,0.,0.,1.], # 1 -> 2 -> 3
    [1.,0.,0.,0.,0.,1.,0.,1.,0.], # 1 -> 3 -> 2
    [0.,1.,0.,1.,0.,0.,0.,0.,1.], # 2 -> 1 -> 3
    [0.,1.,0.,0.,0.,1.,1.,0.,0.], # 2 -> 3 -> 1
    [0.,0.,1.,1.,0.,0.,0.,1.,0.], # 3 -> 1 -> 2
    [0.,0.,1.,0.,1.,0.,1.,0.,0.], # 3 -> 2 -> 1
]


# Find results and probabilities where valid
qaoa_feas_probs = []
for i in range(len(qaoa_result.raw_samples)):
    for j in FEASIBLE_SOLUTIONS:
        if np.array_equal(qaoa_result.raw_samples[i].x, j):
            qaoa_feas_probs.append(qaoa_result.raw_samples[i].probability)


In [69]:
rqaoa = RecursiveMinimumEigenOptimizer(qaoa, min_num_vars=RQAOA_MIN_NUM_VARS, min_num_vars_optimizer=exact)
rqaoa_result = rqaoa.solve(qp)
print(rqaoa_result)

optimal function value: 60.0
optimal value: [0. 0. 1. 1. 1. 0. 0. 0. 0.]
status: SUCCESS


In [96]:
rqaoa_sols=[]
for restart in range(RQAOA_N_RESTARTS):
    rqaoa = RecursiveMinimumEigenOptimizer(
        qaoa, min_num_vars=RQAOA_MIN_NUM_VARS, min_num_vars_optimizer=exact
    )
    rqaoa_result = rqaoa.solve(qp)
    print(rqaoa_result)
    rqaoa_sols.append(rqaoa_result.x)

optimal function value: 120.0
optimal value: [1. 1. 0. 0. 1. 0. 0. 0. 0.]
status: SUCCESS
optimal function value: 120.0
optimal value: [1. 0. 0. 0. 0. 0. 1. 0. 0.]
status: SUCCESS


In [99]:
np.sum(qaoa_feas_probs)

0.109375

In [95]:
rqaoa_res = []
for rqaoa_sol in rqaoa_sols:
    check = any(all(np.array_equal(x,y) for x,y  in zip(sol, rqaoa_sol)) for sol in FEASIBLE_SOLUTIONS)
    rqaoa_res.append(check)
    
rqaoa_prob = sum(rqaoa_res)/len(rqaoa_res)
print(rqaoa_prob)

0.5
